In [1]:
#installing all the modules
!pip install pytubefix
!pip install ffmpeg-python
!pip install faster-whisper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.3/192.3 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.9 MB/s eta 0:00:00


In [2]:
#import pyTube -> Download mp4 from url -> rename removing .mp4
import os
import pytubefix as pytube
url = "https://www.youtube.com/watch?v=DQacCB9tDaw"
yt = pytube.YouTube(url)
yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first().download()

'/content/Introducing GPT-4o.mp4'

In [3]:
os.rename(yt.title + ".mp4", yt.title)

In [4]:
#Extract Audio from the video
import time
import math
import ffmpeg

def extract_audio(input_file):
  extract_audio = f"audio-{input_file}.wav"
  stream = ffmpeg.input(input_file)
  stream = ffmpeg.output(stream, extract_audio)
  ffmpeg.run(stream , overwrite_output=True)
  return extract_audio

In [5]:
audio_extract = extract_audio(yt.title)

Transcribe audio

In [14]:
from faster_whisper import WhisperModel
def transcribe(audio):

    model = WhisperModel("medium")
    segments, info = model.transcribe(audio)
    language = info[0]
    print(f" Transcription Language: {language}")
    segments = list(segments) # this is where the transcribe happens

    for segment in segments:
        print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))
    return language, segments

In [15]:
%%time
language, segments = transcribe(audio_extract)

 Transcription Language: en
[0.00s -> 21.54s]  Hi, everyone. Thank you. Thank you. It's great to have you here today. Today I'm going to
[21.54s -> 28.06s]  talk about three things. That's it. We will start with why it's so important to us to
[28.06s -> 34.70s]  have a product that we can make freely available and broadly available to everyone. And we're
[34.70s -> 40.78s]  always trying to find out ways to reduce friction so everyone can use ChildGPT wherever they
[40.78s -> 48.42s]  are. So today we'll be releasing the desktop version of ChildGPT and the refreshed UI that
[48.42s -> 55.14s]  makes it simpler to use, much more natural as well. But the big news today is that we
[55.14s -> 62.94s]  are launching our new flagship model, and we are calling it GPT-4.0. The special thing about
[62.94s -> 72.26s]  GPT-4.0 is that it brings GPT-4 level intelligence to everyone, including our free users. We'll be
[72.26s -> 78.10s]  showing some live demos today to show the full extent of the 

In [ ]:
filename = 'transcription_segments.txt'
with open(filename, 'w') as file:
    for segment in segments:
        file.write(str(segment.text) + '\n')
print(f'Segments have been written to "{filename}".')

Segments have been written to "transcription_segments.txt".



Subtitle Index : 0, 1, 2

---


Timecode : Start and end markers. HH:MM:SS,sss format.

---

Text : The subtitle text.



In [ ]:
def format_time_for_srt(seconds):
    hours = math.floor(seconds / 3600)
    seconds %= 3600
    minutes = math.floor(seconds / 60)
    seconds %=60
    milliseconds = round((seconds - math.floor(seconds)) * 1000)
    seconds = math.floor(seconds)
    formatted_time = f"{hours :02d}:{minutes:02d}:{seconds:02d},{milliseconds:03d}"

    return formatted_time

In [ ]:
def generate_subtitle_file(input_file, language, segments):
    subtitle_file = f"sub-{input_file}.{language}.srt"
    text = ""
    for index, segment in enumerate(segments):
        segment_start = format_time_for_srt(segment.start)
        segment_end = format_time_for_srt(segment.end)

        text += f"{str(index + 1)}\n"
        text += f"{segment_start} --> {segment_end}\n"
        text += f"{segment.text}\n\n"

    f = open(subtitle_file, "w")
    f.write(text)
    f.close()

    return subtitle_file


In [ ]:
subtitle_file = generate_subtitle_file(yt.title, language, segments)

In [ ]:
def add_subtitle_to_video(input_file, subtitle_file, subtitle_language):
    video_input_stream = ffmpeg.input(input_file)
    subtitle_input_stream = ffmpeg.input(subtitle_file)
    output_video = f"output-{input_file}-{subtitle_language}.mp4"
    subtitle_track_tile = subtitle_file.replace(".srt","")
    stream = ffmpeg.output(video_input_stream, output_video,
                           vf = f"subtitles={subtitle_file}")
    ffmpeg.run(stream, overwrite_output=True)

In [ ]:
language

'en'

In [ ]:
add_subtitle_to_video(yt.title, subtitle_file, language)

**Comparision of Transcripts**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def read_file(filename):
    with open(filename, 'r') as file:
        return file.read()

def compute_cosine_similarity(text1, text2):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([text1, text2])
    similarity_matrix = cosine_similarity(vectors)
    similarity = similarity_matrix[0, 1]
    return similarity

text1 = read_file('transcription_segments(Whispersmall).txt')
text2 = read_file('transcription_segments(WhisperMedium).txt')

similarity = compute_cosine_similarity(text1, text2)
similarity_percentage = similarity * 100

print(f"Similarity Percentage: {similarity_percentage:.2f}%")

Similarity Percentage: 99.55%


In [ ]:
import difflib
from itertools import zip_longest

def read_file(filename):
    with open(filename, 'r') as file:
        return file.read()

def print_non_similar_words(text1, text2):
    words1 = set(text1.split())
    words2 = set(text2.split())
    non_similar_words1 = sorted(words1 - words2)
    non_similar_words2 = sorted(words2 - words1)

    print(f"{'Words in the first transcript but not in the second':<50} | {'Words in the second transcript but not in the first':<50}")
    print('-' * 100)

    for word1, word2 in zip_longest(non_similar_words1, non_similar_words2, fillvalue=''):
        print(f"{word1:<50} | {word2:<50}")

text1 = read_file('transcription_segments(Whispersmall).txt')
text2 = read_file('transcription_segments(WhisperMedium).txt')
print_non_similar_words(text1, text2)


Words in the first transcript but not in the second | Words in the second transcript but not in the first
----------------------------------------------------------------------------------------------------
4.0's                                              | 4                                                 
4O.                                                | 4O,                                               
78,                                                | Absolutely                                        
Always                                             | Ah                                                
Amazing.                                           | Amazing                                           
Awesome.                                           | Bingo.                                            
Barrett,                                           | BurritoJohn78                                     
Bingo!                                             | But,        